load necessary modules

In [1]:
import os
import sys
import numpy as np
import random
import subprocess
import pickle
import torch.nn as nn
import torch
import torch.nn.functional as F
from rwkv.utils import PIPELINE

from MIDI import midi_util
from MIDI.midi_util import VocabConfig,FilterConfig

from model_mjepa import Encoder, Predictor, apply_masks
from model_inference import RWKV
from src.dataset import MyDataset
from src.dataset import process_sequence_array

# Set environment variables
os.environ['RWKV_JIT_ON'] = '1'
# os.environ["RWKV_CUDA_ON"] = '1'
os.environ["RWKV_RESCALE_LAYER"] = '999'


model_path='check_points/finetuned/lora_out_0.pth'


MODEL_FILE = None

EOS_ID = 0
TOKEN_SEP = ' '
TRIAL = 0
ANS_ID=20098


# Load model and tokenizer
# model = RWKV(model=model_path, strategy='cpu fp32')
model = RWKV(model=model_path, strategy='cuda:0 fp16 *65 -> cpu fp32')

pipeline = PIPELINE(model, "json2binidx_tool/tools/tokenizer-midi.json")
tokenizer = pipeline
vocab_config_path="MIDI/vocab_config.json"



RWKV_JIT_ON 1 RWKV_CUDA_ON 0 RESCALE_LAYER 999

Loading check_points/finetuned/lora_out_0.pth ...


/media/bruce/ssd41/zhejing/infilling/code/upload/model_inference.py:209: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.w = torch.load(args.MODEL_NAME, map_location='cpu

Model detected: v4.0
Strategy: (total 64+1=65 layers)
* cuda:0 [float16, float16], store 65 layers
* cpu fp32, store 0 layers
0-cuda:0-float16-float16 1-cuda:0-float16-float16 2-cuda:0-float16-float16 3-cuda:0-float16-float16 4-cuda:0-float16-float16 5-cuda:0-float16-float16 6-cuda:0-float16-float16 7-cuda:0-float16-float16 8-cuda:0-float16-float16 9-cuda:0-float16-float16 10-cuda:0-float16-float16 11-cuda:0-float16-float16 12-cuda:0-float16-float16 13-cuda:0-float16-float16 14-cuda:0-float16-float16 15-cuda:0-float16-float16 16-cuda:0-float16-float16 17-cuda:0-float16-float16 18-cuda:0-float16-float16 19-cuda:0-float16-float16 20-cuda:0-float16-float16 21-cuda:0-float16-float16 22-cuda:0-float16-float16 23-cuda:0-float16-float16 24-cuda:0-float16-float16 25-cuda:0-float16-float16 26-cuda:0-float16-float16 27-cuda:0-float16-float16 28-cuda:0-float16-float16 29-cuda:0-float16-float16 30-cuda:0-float16-float16 31-cuda:0-float16-float16 32-cuda:0-float16-float16 33-cuda:0-float16-float16 

In [2]:

def text_to_midi(input_text, output_path, vocab_config_path,bpm):
    cfg = VocabConfig.from_json(vocab_config_path)
    text = input_text.strip()
    mid = midi_util.convert_str_to_midi(cfg, text,bpm)
    mid.save(os.path.abspath(output_path))

def forward_target(data,masks_pred):
    with torch.no_grad():
        h = jepa_target_encoder(data)
        h = F.layer_norm(h, (h.size(-1),))  # normalize over feature-dim  #1 4094 512
        h = apply_masks(h, masks_pred)   # keep masks_pred
        # h = h.mean(dim=1, keepdim=True)
        return h
def filter_context(a,b):
    # Ensure b is a 1D tensor for easier comparison
    b_flat = b.view(-1)
    
    # Initialize a mask of zeros with the same length as 'a'
    # We'll fill this with ones for elements that should be kept
    mask = torch.ones_like(a, dtype=torch.bool)
    
    # Iterate through each element of 'b_flat' and update the mask
    # Set mask to False for elements in 'a' that are found in 'b_flat'
    for val in b_flat:
        mask &= (a != val)
    
    # Apply the mask to 'a' to filter out unwanted elements
    filtered_a = a[mask]
    
    # Reshape filtered_a back to the original shape if necessary
    filtered_a = filtered_a.view(1, -1)
    
    # Print the result
    return filtered_a

def generate(z1):
    ccc = list(final_sequence[0][:np.where(final_sequence[0]==20097)[0][0]+1])
    gt= list(final_sequence[0][np.where(final_sequence[0]==20097)[0][0]+1:np.where(final_sequence[0]==20098)[0][0]])
    
    occurrence = {}
    state = None
    ccc1=[]
    for i in range(args.ctx_len):
        if i == 0:
            out, state = model.forward(ccc, state,z1)
        else:
            out, state = model.forward([token], state,z1)
    
        # MIDI mode adjustments
        for n in occurrence:
            out[n] -= (0 + occurrence[n] * 0.5)
    
        out[0] += (i - 2000/2) / (500/2)  # not too short, not too long
        if args.patch_size>1000:
            out[20098] += (i - 1000/1) / (500/1)  # not too short, not too long
        else:
            out[20098] += (i - 1000/4) / (500/4)  # not too short, not too long    
        out[127] -= 1  # avoid "t125"
        out[20097] -= 1  
        out[20096] -= 1  
        token = pipeline.sample_logits(out, temperature=1.0, top_k=8, top_p=0.8)
        if token == ANS_ID: 
            print('finish!!!') 
            break
        if token == EOS_ID: break    
        for n in occurrence: occurrence[n] *= 0.997  #### decay repetition penalty
        if token >= 128 or token == 127:
            occurrence[token] = 1 + (occurrence[token] if token in occurrence else 0)
        else:
            occurrence[token] = 0.3 + (occurrence[token] if token in occurrence else 0)
        ccc += [token]
        ccc1+=[token]
    ccc+= [ANS_ID]
    idx=np.where(np.array(ccc)==20096)[0][0] #mask
    idx1=np.where(np.array(ccc)==20097)[0][0] #sep
    new_seq=ccc[:idx]+ccc1+ccc[idx+1:idx1]
    gt_new_seq=ccc[:idx]+gt+ccc[idx+1:idx1]
    patched_new=[list(range(len(ccc[:idx]),(len(ccc[:idx])+len(ccc1))))]
    non_patched_new=[list(range(len(ccc[:idx]))) + list(range(patched_new[0][-1]+1,len(new_seq)))]
    masks_pred_new=torch.tensor(np.array(patched_new))
    masks_enc_new=torch.tensor(np.array(non_patched_new))
    seq='<start>'
    for c in new_seq:
        seq+=' '+tokenizer.decode([c])
    seq+=' <end>'
    
    
    gt_seq='<start>'
    for c in gt_new_seq:
        gt_seq+=' '+tokenizer.decode([c])
    gt_seq+=' <end>'
    data_to_save = np.array([ccc[:idx], ccc1, gt, ccc[idx+1:idx1],tokenizer.encode(seq)], dtype=object)
    text_to_midi(seq,'preference/'+str(j)+'_'+str(ite)+'.mid',"./MIDI/vocab_config.json",120)
    text_to_midi(gt_seq,'preference/gt_'+str(j)+'_'+str(ite)+'.mid',"./MIDI/vocab_config.json",120)
    return masks_pred_new, masks_enc_new,torch.tensor(new_seq[:])
def create_answer_sequence_array(sequence, start_indices, patch_size, sep_token=20098):
    """
    Create the answer sequence from selected patches, separated by `sep_token` using NumPy.
    """
    answer = []
    for start_idx in start_indices:
        patch = sequence[start_idx:start_idx + patch_size]
        answer.extend(patch)
        answer.append(sep_token)
    # Remove the last sep_token
    answer = answer[:]
    return np.array(answer)



def get_non_patched_indices(sequence_length, patched_indices):
    """
    Determine non-patched indices from the sequence based on patched indices.
    """
    non_patched_indices = [i for i in range(sequence_length) if i not in patched_indices]
    return non_patched_indices


def process_sequence_array(sequence, num_patches=1, ctx_len=4096, mask_token=20096, sep_token=20097, ans_token=20098):
    """
    Apply modifications to a single sequence according to specified rules using NumPy.
    Returns the final modified sequence along with patched and non-patched indices.
    """

    patch_size=args.patch_size
    patched_indices_all=[]
    non_patched_indices_all=[]
    final_sequence_all=[]
    for i in range(len(sequence)):
        if len(np.where(sequence[i]==0)[0])==0:
            non_zero=ctx_len-3
        else:
            non_zero=np.where(sequence[i]==0)[0][0]
        # print('non_zero:',non_zero)
        # patch_size = np.random.randint(16, min(non_zero-3,int(ctx_len*0.4)))
        available_index=non_zero-patch_size-1
        if args.fixed_location==1:
            start_indices = 128
        else:
            start_indices = np.random.choice(range(1,available_index))

        patched_indices=list(range(start_indices,start_indices+patch_size))
        non_patched_indices=get_non_patched_indices(len(sequence[i]), patched_indices)
        answer_seq = create_answer_sequence_array(sequence[i], [start_indices], patch_size, sep_token=20098)
        new_sequence=torch.concat([sequence[i][:start_indices],torch.tensor([20096]),sequence[i][start_indices+patch_size:non_zero-1],
             torch.tensor([20097]),torch.tensor(answer_seq),torch.tensor([2])])

        if len(np.where(sequence[i]==0)[0])!=0:
            final_sequence=np.array(torch.concat([new_sequence,torch.zeros(4096-new_sequence.shape[0],dtype=int)]))
        else:
            final_sequence = np.array(new_sequence)
        # print('tessas!',final_sequence.shape)
        final_sequence_all.append(final_sequence)
        patched_indices_all.append(patched_indices)
        non_patched_indices_all.append(non_patched_indices)

    return final_sequence_all, patched_indices_all, non_patched_indices_all
    

In [3]:
class Args:
    def __init__(self):
        # Existing settings
        self.proj_dir = "out"
        self.data_file = "data/pop909_document"
        self.data_type = "binidx"
        self.vocab_size = 20099
        self.ctx_len = 4096
        self.epoch_steps = 100
        self.micro_bsz = 4
        self.embed_dim = 512
        self.encoder_depth=6
        self.predictor_embed_dim=256
        self.predictor_depth=3
        self.predictor_num_heads=4
        self.encoder_num_heads=4      
        self.mlp_ratio = 4
        self.drop_rate=0.1
        self.norm_layer=nn.LayerNorm
        self.init_std=0.02
        self.device= 'cuda:1'
        self.ipe_scale = 1
        self.ema = (0.996, 1.0)
        # New settings from provided args
        self.batch_size = self.micro_bsz
        self.num_workers = 10
        self.pin_mem = True
        self.patch_number= 1

        # Optimization
        self.epochs = 300
        self.final_lr = 1e-6
        self.final_weight_decay = 0.4
        self.lr = 0.001
        self.start_lr = 0.0002
        self.warmup = 40
        self.weight_decay = 0.04
        self.fixed_location=1
        self.patch_size=128
        
args = Args()
train_data = MyDataset(args)


Current vocab size = 20099 (make sure it's correct)
Train Data has 909 samples longer than 512 tokens.


load pre-trained MJEPA

In [4]:

jepa_encoder = Encoder(args)
jepa_predictor = Predictor(args)
jepa_target_encoder= Encoder(args)

checkpoint = torch.load('./check_points/mjepa/jepa-latest.pth.tar', map_location=torch.device(args.device),weights_only=True)

pretrained_dict = checkpoint['encoder']
jepa_encoder.load_state_dict(pretrained_dict)
pretrained_dict = checkpoint['predictor']
jepa_predictor.load_state_dict(pretrained_dict)


pretrained_dict = checkpoint['target_encoder']
jepa_target_encoder.load_state_dict(pretrained_dict)

jepa_target_encoder.eval()
jepa_encoder.eval()
jepa_predictor.eval()

Predictor(
  (predictor_embed): Linear(in_features=512, out_features=256, bias=True)
  (predictor_pos_embed): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (predictor_blocks): ModuleList(
    (0-2): 3 x TransformerEncoderLayer(
      (self_attn): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
      )
      (linear1): Linear(in_features=256, out_features=1024, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (linear2): Linear(in_features=1024, out_features=256, bias=True)
      (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (dropout1): Dropout(p=0.1, inplace=False)
      (dropout2): Dropout(p=0.1, inplace=False)
    )
  )
  (predictor_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  (predictor_proj): Linear(in_features=256, out_features=512, bias=True)
)

HITL finetuning MJEPA based on like and dislike parts

In [5]:
jepa_target_encoder=jepa_target_encoder.to(args.device)
lis = np.random.randint(0, 900, 1)
for j in range(len(lis)):
    print('Start: Music', lis[j],j)
    a=train_data.__getitem__(lis[j])  # original piece   # no indicator tokens
    final_sequence,patched,non_patched=process_sequence_array(a.unsqueeze(0),args.patch_number,len(a)) #randomly initiate dislike part
    masks_pred=torch.tensor(np.array(patched))
    masks_context=torch.tensor(np.array(non_patched))
    with open('./preference/dislike', 'wb') as f:
        pickle.dump([a.unsqueeze(0),masks_pred,masks_context], f)        
    
    for ite in range(1):
        print('Start: Iteration:',j, ite)
        #step 1: select like and dislike
        masks_pred_dislike =masks_pred    
        like_length=min(masks_pred_dislike.shape[1], masks_context.shape[1])            
        like_idx=np.random.choice(masks_context.shape[1]- like_length)
        masks_pred_like=masks_context[0][like_idx:like_idx+like_length].unsqueeze(0)
        masks_context_dislike=filter_context(masks_context,masks_pred_like)
        masks_context_like=masks_context_dislike
        ll=[a.unsqueeze(0),masks_pred_like,masks_pred_dislike,masks_context_like,masks_context_dislike]
        with open('./preference/samples', 'wb') as f:
            pickle.dump(ll, f)
        #step2 finetune JEPA based on like and dislike; iteration-wise
        epochs = 1
        file_path= "./preference/samples"
        log_path= "./preference/"
        if j==0 and ite==0: #intiate mjepa
            print(j,ite,'start !!!')
            load_checkpoint='./check_points/mjepa/jepa-latest.pth.tar'
        else:
            load_checkpoint='./preference/jepa-latest.pth.tar'
        command = f"torchrun --nproc_per_node=1 train_personalization_iteration.py --load_checkpoint {load_checkpoint} --epochs {epochs} --file_path {file_path} --log_folder {log_path}"
        process = subprocess.run(command.split(), capture_output=True, text=True)
        # Output the results
        print(process.stdout)
        print(process.stderr)
        #step3 reload JEPA encoder and generate new music
        checkpoint = torch.load('./preference/jepa-latest.pth.tar', map_location=torch.device(args.device))
        jepa_encoder.load_state_dict(checkpoint['encoder'])
        jepa_predictor.load_state_dict(checkpoint['predictor'])
        jepa_target_encoder.load_state_dict(checkpoint['target_encoder'])
        jepa_encoder=jepa_encoder.to(args.device)
        jepa_target_encoder=jepa_target_encoder.to(args.device)
        jepa_predictor=jepa_predictor.to(args.device)
        jepa_encoder.eval()
        jepa_target_encoder.eval()
        jepa_predictor.eval()
        a=a.to(args.device)
        z = jepa_encoder(a.unsqueeze(0), masks_context_dislike.to(args.device))
        z1 = jepa_predictor(z, a.unsqueeze(0), masks_context_dislike.to(args.device), masks_pred_dislike.to(args.device))
        masks_pred,masks_context,a=generate(z1)
    with open('./preference/like', 'wb') as f:
        pickle.dump([a.unsqueeze(0),masks_pred,masks_context], f)
        
    #step4 finetune model based on user preference: sample-wise, we will get a new JEPA
    epochs = 1
    log_path= "./preference/"
    load_checkpoint='./preference/jepa-latest.pth.tar'
    like_path='./preference/like'
    dislike_path='./preference/dislike'
    command = f"torchrun --nproc_per_node=1 train_personalization_sample.py --load_checkpoint {load_checkpoint} --epochs {epochs} --log_folder {log_path} --like_path {like_path} --dislike_path {dislike_path}"
    process = subprocess.run(command.split(), capture_output=True, text=True)
    # Output the results
    print(process.stdout)
    print(process.stderr)


Start: Music 102 0
Start: Iteration: 0 0
0 0 start !!!
INFO:root:Using AdamW
INFO:root:loaded pretrained encoder from epoch 10 with msg: <All keys matched successfully>
INFO:root:loaded pretrained encoder from epoch 10 with msg: <All keys matched successfully>
['encoder', 'predictor', 'target_encoder', 'opt', 'scaler', 'epoch', 'loss', 'batch_size', 'world_size', 'lr']
INFO:root:loaded pretrained encoder from epoch 10 with msg: <All keys matched successfully>
INFO:root:loaded optimizers from epoch 10
INFO:root:read-path: ./check_points/mjepa/jepa-latest.pth.tar
INFO:root:[1,     1] loss: 0.198 pos and neg: 3.7 3.6 [wd: 4.00e-01] [lr: 1.45e-05] [mem: 2.68e+03] (1207.1 ms)
INFO:root:[1,     1] grad_stats: [0.00e+00 0.00e+00] (1.83e-01, 4.14e+00)
INFO:root:avg. loss 0.198




/tmp/ipykernel_2162089/620473683.py:39: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('./preference/jepa-latest.pth.tar', map_location=torch.device(a

finish!!!
INFO:root:Using AdamW
INFO:root:loaded pretrained encoder from epoch 1 with msg: <All keys matched successfully>
INFO:root:loaded pretrained encoder from epoch 1 with msg: <All keys matched successfully>
['encoder', 'predictor', 'target_encoder', 'opt', 'scaler', 'epoch', 'loss', 'batch_size', 'world_size', 'lr']
INFO:root:loaded pretrained encoder from epoch 1 with msg: <All keys matched successfully>
INFO:root:loaded optimizers from epoch 1
INFO:root:read-path: ./preference/jepa-latest.pth.tar
INFO:root:[1,     1] loss: 7.542 pos and neg: 4.1 3.5 [wd: 4.00e-01] [lr: 1.45e-05] [mem: 1.75e+03] (1048.8 ms)
INFO:root:[1,     1] grad_stats: [0.00e+00 0.00e+00] (1.18e+00, 9.95e+01)
INFO:root:avg. loss 7.542


